In [ ]:
!pip install ipywidgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display

import pandas as pd
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose

import plotly.graph_objects as go
from plotly.subplots import make_subplots

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv("train.csv",
                parse_dates=["Date"],
                index_col="Date")
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.



,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [ ]:
df = df.loc[df['Store']==1]
df['year'] = df.index.year

w = widgets.Dropdown(
    options=df.index.year.value_counts().index.tolist(),
    value=2013,
    description='Task:',
)
year = 2013
def on_change(change):
    global year
    year = 2014

w.observe(on_change)

display(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Dropdown(description='Task:', index=1, options=(2014, 2013, 2015), value=2013)

In [ ]:

data = df.loc[df['year']==year]
data.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,year
Date,,,,,,,,,
2014-12-31,1,3,2605,327,1,0,0,1,2014
2014-12-30,1,2,6466,703,1,0,0,1,2014
2014-12-29,1,1,6463,700,1,0,0,1,2014
2014-12-28,1,7,0,0,0,0,0,1,2014
2014-12-27,1,6,6057,684,1,0,0,1,2014


# **Descriptive statistics with rolling window**

## Time Plot

In [ ]:
layout = go.Layout(plot_bgcolor='#F0E9E6')
fig = go.Figure(layout=layout)

fig.add_trace(
    go.Scatter(
    x=data.index,
    y=data['Sales'],
    line=dict(color='#082255', width=2),
    mode='lines',
    )
)

fig.update_layout(
                   title='Daily Sales for Store 1 for year 2014',
                   xaxis_title='Week',
                   yaxis_title='Sales', 
                   xaxis_showgrid=False, 
                   yaxis_showgrid=False
)

fig.show()

## Rolling Mean

In [ ]:
layout = go.Layout(plot_bgcolor='#F0E9E6')
fig = go.Figure(layout=layout)

fig.add_trace(
    go.Scatter(
    x=data.index,
    y=data['Sales'],
    line=dict(color='#7A95BB', width=2),
    mode='lines',
    name='Sales'
    )
)
fig.add_trace(
    go.Scatter(
    x=data.index,
    y=data['Sales'].rolling(7).mean(),
    line=dict(color='#082255', width=2),
    mode='lines',
    
    )
)

fig.update_layout(
                   title='Daily Sales for Store 1 for year 2014',
                   xaxis_title='Week',
                   yaxis_title='Sales', 
                   xaxis_showgrid=False, 
                   yaxis_showgrid=False
)

fig.show()


## Rolling Mean with Standard Deviations

In [ ]:
rolling_std = data['Sales'].rolling(7).std()

layout = go.Layout(plot_bgcolor='#F0E9E6')
fig = go.Figure(layout=layout)

fig.add_trace(
    go.Scatter(
    x=data.index,
    y=data['Sales'],
    line=dict(color='#7A95BB', width=2),
    mode='lines',
    name='Sales'
    )
)
fig.add_trace(
    go.Scatter(
    x=data.index,
    y=data['Sales'].rolling(7).mean(),
    line=dict(color='#082255', width=2),
    mode='lines',
    
    )
)

fig.add_trace(
    go.Scatter(
            name='Upper Bound',
            x=data.index,
            y=data['Sales'].rolling(7).mean()+(2*rolling_std),
            mode='lines',
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        name='Lower Bound',
        x=data.index,
        y=data['Sales'].rolling(7).mean()-(2*rolling_std),
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
)

fig.update_layout(
                   title='Daily Sales for Store 1 for year 2014',
                   xaxis_title='Week',
                   yaxis_title='Sales', 
                   xaxis_showgrid=False, 
                   yaxis_showgrid=False
)

fig.show()

# **Lag Plots**

In [ ]:
layout = go.Layout(plot_bgcolor='#F0E9E6')
#fig = go.Figure(layout=layout)

fig = make_subplots(
    rows=4, cols=1,
    row_heights=[0.25, 0.25, 0.25, 0.25,],
    #figure=fig,
    row_titles = ['Lag 1','Lag 7','Lag 15','Lag 30']
)
fig.add_trace(
    go.Scatter(
    x=data['Sales'],
    y=data['Sales'].shift(1),
    line=dict(color='#082255', width=2),
    mode='markers',
    name='Lag 1'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
    x=data['Sales'],
    y=data['Sales'].shift(7),
    line=dict(color='#082255', width=2),
    mode='markers',
    name='Lag 7'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
    x=data['Sales'],
    y=data['Sales'].shift(15),
    line=dict(color='#082255', width=2),
    mode='markers',
    name='Lag 15'
    ),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(
    x=data['Sales'],
    y=data['Sales'].shift(30),
    line=dict(color='#082255', width=2),
    mode='markers',
    name='Lag 30'
    ),
    row=4, col=1
)
fig.show()


# **Seasonal Decomposition**

In [ ]:
decomp = seasonal_decompose(data.loc[data['Store']==1]['Sales'],\
                           freq=7)

layout = go.Layout(plot_bgcolor='#F0E9E6')
fig = go.Figure(layout=layout)

fig = make_subplots(
    rows=4, cols=1,
    row_heights=[0.25, 0.25,0.25, 0.25,],
    #figure=fig,
    row_titles = ['Observed','Seasonal','Trend','Residual'])
fig.add_trace(
    go.Scatter(
    x=data.loc[data['Store']==1].index,
    y=decomp.observed,
    line=dict(color='#082255', width=2),
    mode='lines+markers',
    name='Original'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
    x=data.loc[data['Store']==1].index,
    y=decomp.seasonal,
    line=dict(color='#082255', width=2),
    mode='lines+markers',
    name='Seasonal'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
    x=data.loc[data['Store']==1].index,
    y=decomp.trend,
    line=dict(color='#082255', width=2),
    mode='lines+markers',
    name='Trend'
    ),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(
    x=data.loc[data['Store']==1].index,
    y=decomp.resid,
    line=dict(color='#082255', width=2),
    mode='markers',
    name='Resdiuals'
    ),
    row=4, col=1
)
fig.update_layout( 
                   showlegend=False,
                   title='Additive Decomposition',
)

fig.show()